###### As home range of H5 AIV in each month, we combined the virus diffusion in all values of HPD in each month. Corresponding utilization distribution of the virus was the virus diffusion in each value of HPD in every month. For each month slice across a year, we sampled 1000 points at random geographical coordinates within the home range of viruses. 

In [1]:
import json
import pandas as pd
import os
import shutil
import random
import numpy as np
#from geojson_utils import point_in_multipolygon
from shapely import geometry as gt
from shapely import ops
from shapely.prepared import prep

In [2]:
## load a land polygon to test the random points were in land
land = r"../data/world.json"
f1 = open(land)
WORLD_LAND = prep(gt.shape(json.load(f1)['features'][0]['geometry']))
f1.close()

In [17]:
def json2df(file):
    # convert the highest posterior density (HPD) of the inferred virus to dataframe
    f = open(file)
    test = json.load(f)
    f.close()
    df = pd.DataFrame(test['layers'][0]['areas'])
    df.attributes = df['attributes'].apply(lambda x:float(x['HPD']))
    df['corrdinates'] = df['polygon'].apply(lambda cord:
                                                            [list(map(lambda x:
                                                                      [x['xCoordinate'],x['yCoordinate']],cord['coordinates']
                                                                     )
                                                                 )
                                                            ])
    df.drop('polygon',axis = 1,inplace = True)
    df['polygon']=df.corrdinates.apply(lambda x:gt.shape({"type":"MultiPolygon","coordinates":[x]}))
    
    groups = df.groupby(['startTime','attributes'])
    
    out_df = pd.DataFrame(groups.apply(lambda x:ops.unary_union(list(x.polygon))),columns=['polygon'])
    out_df_full = pd.DataFrame(ops.unary_union(list(df.polygon)))
    
    return out_df_full

In [13]:
def get_random(convex_hull,num):
    '''
    convex_hull:  home range of the Mulitpolygon in one time(month)
    out: points within conver_hull and land
    '''
    #print(union_polygon)
    #num = 100
    polygon_shp = convex_hull['polygon']
    prep_shp = prep(polygon_shp)
    #stime = str(convex_hull.name)
    minx, miny, maxx, maxy = polygon_shp.bounds
    
    tlist_of_points = []
    counter = 0
    while counter < num:
        pnt = gt.Point(random.uniform(minx, maxx), random.uniform(miny, maxy))
        if prep_shp.contains(pnt) and WORLD_LAND.contains(pnt):
            tlist_of_points.append([pnt,pnt.coords[:][0][1],pnt.coords[:][0][0]])
            counter += 1 
    out = pd.DataFrame(tlist_of_points,columns=['point','lat','lon'])
    ## qiqi
    out.loc[:,'month']=1
    out_months=out
    for k in range(2,13):
        out_tmp=out
        out_tmp.loc[:,'month']=k
        out_months=pd.concat([out_months,out_tmp])
        
    #out_final = pd.concat(out_months)
    return out_months

In [14]:
def inclusion_relation(point,polygons):
    '''
    # point - one random point in one time
    # polygon - virus polygons, polygons.loc[point.startTime] get the polygon in this time
    # out - the spatial relationship between point and these polygons ,and are randed by 0.1-0.95HPD
    '''
    out = list(map(lambda x:x.contains(point.point),polygons.loc[point.month]['polygon']))
    return out

In [15]:
def stream(virus_path,num,out_path):

    df_v = json2df(virus_path)
    #df_v = df_v.reset_index(level='attributes')
    
    #full = ops.unary_union(df_v[1,'polygon'],df_v[2,'polygon'])
    
    ## get the convex_hull of every timesilces
    #time_group = df_v.groupby('month')
    #time_hull = pd.DataFrame(time_group.apply(lambda x:ops.unary_union(list(x.polygon))),columns = ['convex_hull'])
    #full = pd.concat(df_v)
    df_v.to_csv('../data/df_v.csv')
    #full_region = ops.unary_union(pd.concat(df_v['polygon']))
    #full_region = pd.DataFrame(df_v.apply(lambda x:ops.unary_union(list(x.polygon))))
    random_df = pd.concat(df_v.apply(get_random,axis=1,**{'num':num}).to_list()).reset_index(drop=True)
    
    #random_df = pd.concat(df_v.apply(get_random,axis=1,**{'num':num}).to_list()).reset_index(drop=True)
    #random_df.to_csv('../data/random_points.csv')
    # The spatial relationship between random points and each HPD 
    spatial_relate = random_df.apply(inclusion_relation,axis=1,**{'polygons':df_v})
    hpd_l = list(np.arange(10,100,5)/100)
    spatial_relate = pd.DataFrame(list(map(lambda x:x, spatial_relate)),columns=hpd_l)
    random_relate = pd.concat([random_df,spatial_relate],axis=1).set_index('month')
    
    random_relate.to_csv(out_path)

In [18]:
virus_file1 = r'../data/HPD_10_95_2.3.2.1.json'
outfile1 = r'../data/random_df_2.3.2.1_noConvexHull.csv'
random_num1 = 5
stream(virus_path=virus_file1,num=random_num1,out_path=outfile1)

/Users/qiqiy/anaconda3/envs/jupyterlab-debugger/lib/python3.10/site-packages/pandas/core/dtypes/cast.py:1981: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  result[:] = values
/Users/qiqiy/anaconda3/envs/jupyterlab-debugger/lib/python3.10/site-packages/pandas/core/frame.py:710: ShapelyDeprecationWarning: __len__ for multi-part geometries is deprecated and will be removed in Shapely 2.0. Check the length of the `geoms` property instead to get the  number of parts of a multi-part geometry.
  data = list(data)
/Users/qiqiy/anaconda3/envs/jupyterlab-debugger/lib/python3.10/site-packages/pandas/core/frame.py:710: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  data = list(data

KeyError: 'polygon'

In [ ]:
virus_file2 = r'../data/HPD_10_95_2.3.4.4.json'
outfile2 = r'../data/random_df_2.3.4.4_noConvexHull.csv'
random_num2 = 1000
stream(virus_path=virus_file2,num=random_num2,out_path=outfile2)